In [1]:
from carbon.helpers.cryptocompare import CryptoCompare
import pandas as pd
from matplotlib import pyplot as plt
import os
print( "{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CryptoCompare))

CryptoCompare v1.3 (30/Jan/2023)


# Crypto Compare

https://min-api.cryptocompare.com/documentation

## SetUp

If you have a private API key, run `export CCAPIKEY=<YOURKEY>` before you launch Jupyter on the same shell. In case no key is found the free API is used. You can also provide the API key in the constructor, but this is not recommended for security reasons.

In [2]:
try:
    CC = CryptoCompare(apikeyname="CCAPIKEY")
except:
    CC = CryptoCompare(apikey=True)

print("[CryptoCompare] key digest", CC.keydigest[:4])

Can't find API key CCAPIKEY in environment variables.
Use `export CCAPIKEY=<value>` to set it BEFORE you launch Jupyter
[CryptoCompare] key digest 0000


## Coin lists

Here we populate the various coin lists that we will use to download data. They are as follows:

- `coins_for_ccy`, `ccies`: creates a table for each of the currencies 
- `coins_for_cross`: creates a single cross table
- `usdstables`: creates a single table against USD

We also define the list of items **not** to produce, `excludes`. Finally there is an `includes` list that, if present, specifies the _only_ items to be produced.

In [3]:
coins_for_ccy ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
coins_for_cross ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
usdstables = "USDT, USDC, UST, BUSD"
ccies = "USD, BTC, ETH"

#excludes = ['COINS-CROSS', 'STABLES-USD', 'COINS-USD', 'COINS-BTC', 'COINS-ETH']
includes = ["STABLES-USD"]

The table `dltable0` has as keys the filename, and the data is a tuple of pairs. The table `dltable` contains the final downloads, the difference being the `excludes` to avoid redownloading data that is not needed. 

**YOU MUST RESTART THE KERNEL IF YOU MAKE CHANGES TO INCLUDES OR EXCLUDES**.

In [4]:
dltable0 = {
    "COINS-CROSS": CC.create_pairs(coins_for_cross),
    "STABLES-USD": CC.create_pairs(usdstables, "USD"),
    **{
        f"COINS-{ccy}": CC.create_pairs(coins_for_ccy, ccy)
        for ccy in CC.coinlist(ccies)
    }
}
try:
    dltable = {k:v for k, v in dltable0.items() if k in includes}
    print("restricted execution to the pairs in `includes`", includes)
except:
    try:
        dltable = {k:v for k, v in dltable0.items() if not k in excludes}
        print("the items in `excludes` were excluded", excludes)
    except:
        dltable = dltable0
        print("no `includes` or `excludes` info present; all tables are being process")
dltable0.keys(), dltable.keys()

restricted execution to the pairs in `includes` ['STABLES-USD']


(dict_keys(['COINS-CROSS', 'STABLES-USD', 'COINS-USD', 'COINS-BTC', 'COINS-ETH']),
 dict_keys(['STABLES-USD']))

## Data download

### Raw tables

In [5]:
for item, pairs in dltable.items():
    print("Downloading raw table", item, len(pairs))
    results = {
        (fsym, tsym): CC.query_dailypair(fsym=fsym, tsym=tsym)
        for fsym, tsym in pairs
    }
    df = pd.concat(results, axis=1)
    df.to_pickle(CC.datafn(f"{item}-RAW.pickle"))

[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'USDT', 'tsym': 'USD', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'USDC', 'tsym': 'USD', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'UST', 'tsym': 'USD', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'BUSD', 'tsym': 'USD', 'limit': 2000} [0000]


In [6]:
!ls cryptocompare

COINS-BTC-RAW.pickle      COINS-ETH.pickle          isoccy.csv
COINS-BTC.pickle          COINS-USD-RAW.pickle      query_allexchanges.pickle
COINS-CROSS-RAW.pickle    COINS-USD.pickle          query_coinlist.pickle
COINS-CROSS.pickle        STABLES-USD-RAW.pickle    query_indexlist.pickle
COINS-ETH-RAW.pickle      STABLES-USD.pickle


### Reformatted tables

In [7]:
for item in dltable:
    print("Converting raw table", item)
    df = pd.read_pickle(CC.datafn(f"{item}-RAW.pickle"))
    df = CC.reformat_raw_df(df)
    df.to_pickle(CC.datafn(f"{item}.pickle"))

Converting raw table STABLES-USD


## Review tables

In [8]:
!ls cryptocompare

COINS-BTC-RAW.pickle      COINS-ETH.pickle          isoccy.csv
COINS-BTC.pickle          COINS-USD-RAW.pickle      query_allexchanges.pickle
COINS-CROSS-RAW.pickle    COINS-USD.pickle          query_coinlist.pickle
COINS-CROSS.pickle        STABLES-USD-RAW.pickle    query_indexlist.pickle
COINS-ETH-RAW.pickle      STABLES-USD.pickle


In [9]:
df = pd.read_pickle("cryptocompare/COINS-ETH.pickle")
df = pd.read_pickle("cryptocompare/COINS-CROSS.pickle")
df

,ETH/OKB,ATOM/DOGE,OKB/BNB,SOL/ADA,DOT/XRP,DOGE/BTC,LTC/XRP,DOGE/DOT,SOL/DOT,ETH/SOL,...,ADA/XRP,SOL/XRP,LTC/ETH,LTC/DOGE,AVAX/BTC,AVAX/DOT,AVAX/ADA,SOL/BTC,BNB/BTC,ATOM/ETH
datetime,,,,,,,,,,,,,,,,,,,,,
2017-08-09,0.00,0.00,0.0000,0.00,0.00,5.800000e-07,258.48,0.00000,0.000,0.00,...,0.0000,0.00,0.16140,24568.97,0.000000,0.000,0.00,0.000000,0.00000,0.000000
2017-08-10,0.00,0.00,0.0000,0.00,0.00,5.500000e-07,259.18,0.00000,0.000,0.00,...,0.0000,0.00,0.15750,24909.09,0.000000,0.000,0.00,0.000000,0.00000,0.000000
2017-08-11,0.00,0.00,0.0000,0.00,0.00,5.100000e-07,264.00,0.00000,0.000,0.00,...,0.0000,0.00,0.15190,25235.29,0.000000,0.000,0.00,0.000000,0.00000,0.000000
2017-08-12,0.00,0.00,0.0000,0.00,0.00,4.700000e-07,268.49,0.00000,0.000,0.00,...,0.0000,0.00,0.14960,25340.43,0.000000,0.000,0.00,0.000000,0.00000,0.000000
2017-08-13,0.00,0.00,0.0000,0.00,0.00,4.400000e-07,273.24,0.00000,0.000,0.00,...,0.0000,0.00,0.15260,25318.18,0.000000,0.000,0.00,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26,44.63,152.03,0.1178,64.08,15.80,3.750000e-06,213.71,0.01333,3.761,65.80,...,0.9275,59.44,0.05463,1014.40,0.000786,2.795,47.62,0.001058,0.01324,0.008180
2023-01-27,44.14,155.05,0.1175,62.74,15.93,3.780000e-06,215.20,0.01325,3.713,65.40,...,0.9430,59.16,0.05562,1019.05,0.000919,3.223,54.45,0.001059,0.01335,0.008463
2023-01-28,39.96,150.29,0.1285,62.87,15.75,3.820000e-06,220.10,0.01370,3.750,65.48,...,0.9368,58.89,0.05702,1020.42,0.000894,3.214,53.88,0.001043,0.01330,0.008404
